In [253]:
import json, requests
import pandas as pd
import numpy as np

def getdata(initiallink):
    
    link=initiallink
    data=pd.Series(dtype="float64")
    morenextlinks=True
    
    while(morenextlinks):

        try:
            #get the data
            datarequest=json.loads(requests.get(link).text)
            if(datarequest['value']==[]):
                #print('no observations under this link')
                break

        except: 
            #print('no observations under this link')
            break


        if(len(datarequest['value'])>0):
            #append chunks
            datachunk=pd.Series(list(map(lambda x: x["result"],datarequest["value"])),index=list(map(lambda x: pd.to_datetime(x["resultTime"]),datarequest["value"])))
            data=data.append(datachunk)

            try:
                link=datarequest["@iot.nextLink"]
            except:
                morenextlinks=False;
                break
    return data


def handle(event, context):
    
    t=5
    
    url = "https://api.smartaq.net/v1.0"
    frostquery="/Datastreams('saqn%3Ads%3Afc82203')/Observations?$filter=phenomenonTime%20gt%202020-05-20T02:00:00.000Z%20and%20phenomenonTime%20lt%202020-05-22T02:00:00.000Z"
    
    freq=str(t) + "T"
    
    data=getdata(url + frostquery)
    
    #check for empty link
    try:
        data[0]
    except: 
        return {
            "statusCode": 404,
            "body": {}, 
            "headers": {
                "Content-Type": "application/json"
            }
        }
    
    #aggregate the data
    agg=data.resample(freq).agg([np.sum, np.mean, np.std])
    agg["count"]=(agg["sum"]/agg["mean"]).apply(int)
    
    #output the result as a dataframe
    result=pd.DataFrame({"result": list(agg["mean"]),"resultTime": list(agg.index.map(lambda x: x.isoformat().split("+")[0] + ".000Z")),"phenomenonTime": list(agg.index.map(lambda x: (x-pd.Timedelta(str(t) + "m")).isoformat().split("+")[0] + ".000Z" + "/" + x.isoformat().split("+")[0] + ".000Z")), "parameters": json.loads(agg[["count","std"]].to_json(orient="records"))})
    
    
    
    return {
        "statusCode": 200,
        "body": result.to_json(orient='records'), 
        "headers": {
            "Content-Type": "application/json"
        }
    }



In [254]:
test=handle("a","b")["body"]

In [258]:
json.loads(test)[-1]

{'result': 963.7095909091,
 'resultTime': '2020-05-22T01:55:00.000Z',
 'phenomenonTime': '2020-05-22T01:50:00.000Z/2020-05-22T01:55:00.000Z',
 'parameters': {'count': 22, 'std': 0.0212951125}}

In [256]:
len(json.loads(test))

551

In [209]:
.append(test[:2].append(test[2:4]))

2020-05-20 04:06:50+00:00    961.2008
2020-05-20 04:07:01+00:00    961.2252
2020-05-20 04:07:11+00:00    961.1922
2020-05-20 04:07:21+00:00    961.1848
dtype: float64

In [236]:
pd.Series([1,2,3],dtype="float64")[0]

1.0

In [241]:
    url = "https://api.smartaq.net/v1.0"
    frostquery="/Datastreams('saqn%3Ads%3Afc82203')/Observations?$filter=phenomenonTime%20gt%202020-05-20T02:00:00.000Z%20and%20phenomenonTime%20lt%202020-05-20T09:00:00.000Z"
    
    freq=str(t) + "T"
    
    getdata(url + frostquery)


2020-05-20 04:06:50+00:00    961.2008
2020-05-20 04:07:01+00:00    961.2252
2020-05-20 04:07:11+00:00    961.1922
2020-05-20 04:07:21+00:00    961.1848
2020-05-20 04:07:31+00:00    961.2009
                               ...   
2020-05-20 08:59:15+00:00    962.2452
2020-05-20 08:59:25+00:00    962.2336
2020-05-20 08:59:36+00:00    962.2336
2020-05-20 08:59:46+00:00    962.2036
2020-05-20 08:59:56+00:00    962.2459
Length: 1388, dtype: float64

In [221]:
data

Series([], dtype: float64)